# Část 2
Nyní zkusíme využít oklasifikované události z předchozí části a upravit jimi znalostní bázi našeho LLM modelu.

Nezapomeňte si opět nainstalovat balíček _snowflake-ml-python_ z tlačítka _Packages_ v horní liště.

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.snowpark.functions import col

from snowflake.cortex import Complete, CompleteOptions

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
# Pomocne fce
import json
import random

# Pomocna funkce na prevedeni radku do JSON formatu
def jsonify_row(row):
    return json.dumps(row.as_dict(), indent=2, ensure_ascii=False)


# Ocisteni uzivatelskeho jmena
def user_suffix(session):
    if name := session.get_current_user():
        return name[1:-1]
    # nemelo by nastat, ale at mame string
    return f"NONE_{random.randint(1,1000)}"

In [ ]:
model = "llama3.1-70b"
user_tables_suffix = user_suffix(session)
categorization_table = f"EVENT_CATEGORIZATION_{user_tables_suffix}"

df_categorization = session.read.table(categorization_table)
df_test_questions = session.read.table("TEST_2024")
test_q_collected = df_test_questions.collect()

In [ ]:
st.dataframe(df_test_questions)

In [ ]:
def get_random_test_question():
    idx = random.randint(0, len(test_q_collected) - 1)
    return test_q_collected[idx]


def get_question(idx):
    return test_q_collected[idx]

In [ ]:
# Zkuste si, jake odpovedi nam dava model na otazky z roku 2024
# Experimentujte take s parametrem temperature

question = get_random_test_question()  # Nebo si ulozte jednu predem a upravujte temperature
question = get_question(1)

system_prompt = "You are an assistant specialized in history. Provide short and accurate answers to user's questions."
user_prompt = question.Q_TEXT
expected_answer = question.Q_ANSWER

options = CompleteOptions(temperature=0.8)

classification = Complete(
    model,
    [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    options=options
)

print(f"""Question to model: {user_prompt}
Expected answer: {expected_answer}
Model answer: {classification}
""")

In [ ]:
-- Mimochodem ve Snowflake noteboocich je mozne kombinovat Python a SQL
-- Volani Complete fce v SQL muze vypadat treba takto
SELECT SNOWFLAKE.CORTEX.COMPLETE(
    'llama3.1-8b',
    ARRAY_CONSTRUCT(
        OBJECT_CONSTRUCT(
            'role', 'system', 
            'content', 'You are a helpful assisstant trying to answer user''s questions'),
        OBJECT_CONSTRUCT(
            'role', 'user', 
            'content', 'When is your trainig data cut-off?')
    ),
    OBJECT_CONSTRUCT(
        'temperature', 0.1
    )
);

## Úloha - naivní RAG
A nyní zkusíme využít námi oklasifikované události z roku 2024, abychom rozšířili znalosti námi používaného LLM modelu.

V první fázi zkusíme, jestli dostaneme očekávané výsledky na události z roku 2024 ve sportovní kategorii.

In [ ]:
only_sport_q = df_test_questions.filter(col("CATEGORY_ID") == 2)
only_sport_events = df_categorization.filter(col("CATEGORY_ID") == 2)

In [ ]:
only_sport_events

In [ ]:
info = "/n".join(jsonify_row(r) for r in only_sport_events.select("EVENT_TEXT").collect())

system_prompt = f"""You are a helpful assistant. Use short answers, at most 5 words long. When answering user's question, use following information (each information is a JSON object with element EVENT_TEXT describing the event):
{info}
"""

# Klasiciky si zkuste i ruzne hodnoty temperature
options = CompleteOptions(temperature=0.1)

sports_list = []
for row in only_sport_q.collect():
    answer = Complete(
        model,
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": row.Q_TEXT},
        ],
        options=options
    )
    sports_list.append(
        [row.Q_TEXT, row.Q_ANSWER, answer]
    )

sport_df = session.create_dataframe(sports_list, ["question", "expected", "llm_answer"])

In [ ]:
st.dataframe(sport_df)

Nyní se pokuste seskládat všechny výše použité kódy k tomu, abyste sestavili _vyhodnocovacího agenta_. Ten bude mít podobu funkce (viz níže), která na vstupu dostane uživatelský prompt. Našim cílem bude kategorizovat uživatelovu otázku, na základě toho vybrat události pouze z relevantní kategorie a následně odpovědět.

Pokud uživatel zadá otázku mimo naše 4 kategorie, buď mu řekněte, že na takové otázky neodpovídáte, nebo nechcte odpovědět LLM bez poskytnutí znalostní báze.

\* Pozn.: Kontextové okno llamy je dost velké, aby se do něj vešly všechny naše kategorizované události. Toto řešení ale nechceme ;) 

In [ ]:
def perform_naive_rag(user_prompt: str) -> str:
    # Faze 1 - kategorizujme uzivatelovu otazku
    system_prompt_1 = "Classify user prompt..."
    category = Complete(...)  # mozna bude nutne jeste nejak transformovat

    # Vybereme kategorii znalostni baze dle klasifikace user promptu
    relevant_documents = (
        df_categorization
        .filter(col("CATEGORY") == category)
    )

    system_prompt_2 = "..."   
    # Odpovime na uzivatelovu otazku
    answer = Complete(...)
    
    return answer

In [ ]:
# Vyzkousime si odpovedi ...
test_questions = random.sample(df_test_questions.collect(), k=10)

naive_rag_ans = []
for tq in test_questions:
    llm_answer = perform_naive_rag(tq.Q_TEXT)
    naive_rag_ans.append([tq.Q_TEXT, tq.Q_ANSWER, llm_answer])

naive_rag_df = session.create_dataframe(naive_rag_ans, ["question", "expected", "llm_answer"])

V reálné praxi samozřejmě máme znalostní bázi podstatně větší a potřebujeme tedy používat efektivnější metody přístupu k ní. Typicky každý dokument, který chceme zařadit do znalostní báze, rozdělíme na menší části, nad kterými následně spočítáme _embedding_ (transformujeme text do číselného vektoru).

Dokumenty včetně embeddingů následně uložíme do _vektorové databáze_, od které očekáváme, že dokáže efektivně vyhledat _n_ nejbližších dokumentů na základě podobnosti _embeddingů_. 

V další fázi si tedy zkusíme spočítat embeddingy nad našimi událostmi a vyhledávat mezi nimi na základě našich dotazů.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# Snowflake embedding modely maji limit 512 tokenu, takze je potreba nase texty rozdelit
# Muzete zkusit experimentovat s parametry nebo jinymi splittery
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=400,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

all_event_texts = df_categorization.select("EVENT_TEXT").collect()
texts = text_splitter.create_documents([r.EVENT_TEXT for r in all_event_texts])

In [ ]:
from snowflake.cortex import EmbedText768, EmbedText1024


# Zaexperimentujte si s ruznymi modely a delkou vektoru 
embedding_model = "snowflake-arctic-embed-m"  
# embedding_model = "e5-base-v2"

vector_table = f"VECTOR_{user_tables_suffix}"

chunks_df = session.create_dataframe([t.page_content for t in texts], ["text"])
chunks_embedded = chunks_df.withColumn(
    "embedding", 
    EmbedText768(embedding_model, chunks_df["TEXT"])
)
chunks_embedded.write.save_as_table(vector_table, mode='overwrite')


In [ ]:
# Vyhledavani podle kosinove vzdalenosti bohuzel jeste neni v Python API, takze si musime 
# pomoct takto:
user_query = "How many states did Kamala Harris win in 2024?"
emb_limit = 3
df = session.sql(f"""
    SELECT
        t.text,
        VECTOR_COSINE_SIMILARITY(
            t.embedding,
            SNOWFLAKE.CORTEX.EMBED_TEXT_768(
                ?, 
                ?
            )
        ) AS similarity
    FROM {vector_table} AS t
    ORDER BY similarity DESC
    LIMIT {emb_limit}
    """,
    params=[embedding_model, user_query]            
)

In [ ]:
df

Nyní vytvořte nového agenta analogického k _perform_naive_rag_, který ovšem pro vyhledávání relevantních dokumentů budou používat před chvílí vytvořenou tabulku s embeddingy jednotlivých událostí. 

Experimentuje s různými modely, velikostmi embeddingů a počtem dokumentů, které do modelu dodáte.

In [ ]:
def embedding_rag(user_prompt):
    pass  # Now it's your task ;)

In [ ]:
# Vyzkousime si odpovedi ...
rag_test_questions = random.sample(df_test_questions.collect(), k=10)

embedding_rag = []
for tq in rag_test_questions:
    llm_answer = perform_naive_rag(tq.Q_TEXT)
    embedding_rag.append([tq.Q_TEXT, tq.Q_ANSWER, llm_answer])

embedding_rag_df = session.create_dataframe(embedding_rag, ["question", "expected", "llm_answer"])

In [ ]:
embedding_rag_df.show()

## Bonus
Vytvořte LLM, který _lže_. Vytvořte si vlastní znalostní bázi, která bude obsahovat nepravdivé informace, a pokuste se s jejím využitím nastavit systémový prompt tak, aby model bez zaváhání odpovídal nepravdivě na všeobecně známa historická fakta (přesné otázky si vymyslete sami, nebo si témata nechte vygenerovat ;), ale jako inspiraci dáváme):

- Samostatná Česká republika nevznikla 1.1.1993
- Prvním člověkem ve vesmíru byl Jára Cimrman
- Americkou občanskou válku vyhrála Konfederace
- Richard Nixon vyhrál prezidentské volby už v roce 1960 a J.F. Kennedy tak zůstal až do své smrti roku 2012 řadovým senátorem za Massachusetts
- Otcem Luka Skywalkera je Obi-Wan Kenobi

Zajistěte, aby LLM podávalo odpovědi jako fakta, kterým opravdu věří. Především zamezte tomu, aby odpovídalo stylem "Podle mé znalostní báze se to sice stalo takhle, ale to je špatně a ve skutečnosti ...".